In [ ]:
!pip install sentencepiece
!pip install transformers
import torch
# from transformers import AutoTokenizer, AutoModel, pipeline
# import re
from transformers import T5Tokenizer, T5EncoderModel
import numpy as np
import os
# import requests
from tqdm.auto import tqdm
#from numba import cuda
import gc

In [ ]:
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()
device = torch.device("cuda:0")
# device = torch.device("cpu")
# model.to(device)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc", do_lower_case = False)
# tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case = False)

In [ ]:
# model = AutoModel.from_pretrained("Rostlab/prot_bert") # load the model PROTBERT
# model = T5Model.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")

In [ ]:
model.to(device)

In [ ]:
import pandas as pd
path =  '/kaggle/input/final-data/'

In [ ]:
succinylation_negative = pd.read_csv(path + 'negative_sites_sequence.csv').dropna().drop_duplicates()
succinylation_positive = pd.read_csv(path + 'positive_sites_sequence.csv').dropna().drop_duplicates()

In [ ]:
succinylation_positive.shape

In [ ]:
positive_datasets = {}
negative_datasets = {}
positive_datasets['protein_position'] = succinylation_positive['site']
positive_datasets['protein_seq'] = succinylation_positive['protein_sequence']
negative_datasets['protein_position'] = succinylation_negative['site']
negative_datasets['protein_seq'] = succinylation_negative['protein_sequence']


In [ ]:
window_size = 15 # Actual : 100 + 1 + 100
def extract_embedding(encoded_input):
  embedding = None  
  with torch.no_grad():
    embedding = model(**encoded_input)[0]
    embeds = embedding.detach().to('cpu')
    del embedding
    torch.cuda.empty_cache()
    gc.collect()

  return embeds

from copy import deepcopy
def clip_features(embedding, sequence, position):
  sequence = sequence.replace(' ','')
  prot_pos = position-1 
  feature = embedding[prot_pos-window_size: prot_pos+window_size+1]
  return feature, position


def generate_embeddings(sequences, positions):
  encoded_input = None
  embedding = None
  generated_embeddings = []
  new_positions = []
  for i, sequence in enumerate(sequences):
#     if i < 430:continue
    if i % 100 == 0: print("Batch_Size: ",i)
    position = positions[i]
    if position > len(sequence): continue
    if len(sequence) > 4900:
        if position < 4900:
            sequence = sequence[:4900-1]
        else:
            sequence = sequence[4900-1:]
            position = position-4900+1       
            
    new_sequence = ''
    prot_pos = position-1            
    
    if prot_pos - window_size < 0 and len(sequence[prot_pos+1:]) < window_size:
        new_sequence = 'X'*(window_size-prot_pos) + sequence + 'X'*(window_size-len(sequence[prot_pos+1:])) 
        position = 16
    elif prot_pos - window_size < 0:
#         print("num",prot_pos, window_size)
        new_sequence = 'X'*(window_size-prot_pos) + sequence
        position = 16
    elif len(sequence[prot_pos+1:]) < window_size:
        new_sequence = sequence + 'X'*(window_size-len(sequence[prot_pos+1:])) 
    else:
        new_sequence = sequence
    sequence = " ".join(new_sequence) #Adding spacing between each character as rostlab takes spaces between each word
#     print("sequence_length: ", len(new_sequence))
    encoded_input = tokenizer(sequence,return_tensors='pt').to(device)
#     encoded_input = tokenizer.batch_encode_plus(sequence, add_special_tokens=True, padding="longest")
    embedding = extract_embedding(encoded_input)
    embedding =feature_extractor(embedding, sequence)
#     print(embedding.shape)
    ##### Write a code to extract 200 positions 
    feature,position = clip_features(embedding, sequence, position)
#     print(embedding.shape, feature.shape[0], i, position)
    assert feature.shape[0] == 31
    generated_embeddings.append(feature.numpy())
    new_positions.append(position)
    del encoded_input
    del embedding                 
    torch.cuda.empty_cache()
    gc.collect()   

  return generated_embeddings, new_positions


def feature_extractor(embedding, sequence):
  seq_emb = embedding
#   print(seq_emb[0].shape)
  seq_emb = seq_emb[0][:len(sequence)] 
  return seq_emb


def save_file(filename, features):
    path = '/kaggle/working/'
#     filename = 'negative.npy'
#     torch.save(features, path+filename)
    np.savez(path + filename, features)



In [ ]:
import math
neg_sequences = list(negative_datasets['protein_seq'])
neg_sequences_position = list(negative_datasets['protein_position'])
no_of_iter = math.ceil(len(neg_sequences)/500)
neg_pos_list = []
for i in range(no_of_iter):
    print("No_of_iterations:", i+1)
    neg_embeddings, neg_position = generate_embeddings(neg_sequences[:500], neg_sequences_position[:500])
    neg_pos_list.extend(neg_position)
    save_file('negative'+str(i)+'.npz', neg_embeddings)
    neg_sequences = neg_sequences[500:]
    neg_sequences_position = neg_sequences_position[500:]

In [ ]:
import math
pos_sequences = list(positive_datasets['protein_seq'])
pos_sequences_position = list(positive_datasets['protein_position'])
no_of_iter = math.ceil(len(pos_sequences)/500)
pos_pos_list = []
for i in range(no_of_iter):
    print("No_of_iterations:", i+1)
    pos_embeddings, pos_position = generate_embeddings(pos_sequences[:500], pos_sequences_position[:500])
    pos_pos_list.extend(pos_position)
    pos_sequences = pos_sequences[500:]
    pos_sequences_position = pos_sequences_position[500:]
    save_file('positive'+str(i)+'.npz', pos_embeddings)
    

In [ ]:
path = '/kaggle/working/'
df = pd.DataFrame({'position': pos_pos_list})
df.to_csv(path+'positive_position.csv', index=False)